# Actual stuff

In [ ]:
# get some basic os functions
import os
import glob
# import our parallel processing framework
import dask
import dask.multiprocessing
import distributed
# numpy and plotting
import numpy as np
from itertools import accumulate
from operator import matmul
%matplotlib inline
import matplotlib.pyplot as plt
# image reading and righting
from skimage.external import tifffile as tif
# load our actual registration functions
%load_ext autoreload
%autoreload 2
from imreg_dph import register_ECC, similarity, translation, register_translation

if running on the cluster we need to start our scheduler:

    dask-scheduler --scheduler-file /groups/betzig/home/hoffmand/dask-scheduler.json

this should be done on the same computer that's running this notebook (not windows, login1 or login2 or
 a qlogin interactive session on the cluster)

In [ ]:
# if running on cluster we need to start our workers
!qsub -pe batch 16 -l sandy=true -t 1-64 -b y -j y -o ./dask-log.log -cwd -V dask-worker --scheduler-file /groups/betzig/home/hoffmand/dask-scheduler.json

In [ ]:
# if running on the cluster we need to attached to the scheduler
client = Client(scheduler_file="/groups/betzig/home/hoffmand/dask-scheduler.json")

In [ ]:
# change this if needed
paths = sorted(glob.iglob("/groups/hess/hesslab/Cryo_data/EM_Data/17-7_20170220_CS2_Cell6/Cryo_20170220_*_InLens/*.tif"))
len(paths)

In [ ]:
# generate a list of pairs to register together
pairs = [(paths[i], paths[i + 1]) for i in range(len(paths) - 1)]
pairs[:4]

In [ ]:
def load_and_register(path0, path1, method=register_translation, *args, **kwargs):
    """Load two images defined by path0 and path1 (assumed to be tif) and register with meth
    
    Parameters
    ----------
    path0 : URI to data0
    path1 : URI to data1
    method : callable
        the registration algortithm
    args : args passed to meth
    kwargs : kwargs passed to meth
    
    Returns
    -------
    af : AffineTransformation
        The affine transformation that registers path1 to path0
    """
    data0 = tif.imread(path0)
    data1 = tif.imread(path1)
    
    return method(data0, data1, *args, **kwargs)

In [ ]:
# test the registration and time it
%time af = load_and_register(*pairs[0])
af

In [ ]:
# as an example let's compute using ecc with translation and rotation
%time to_compute = dask.delayed([dask.delayed(load_and_register, pure=True)(path0, path1, register_ECC, warp_mode=cv2.MOTION_EUCLIDEAN, num_iter=500, term_eps=1e-6) for path0, path1 in pairs])
%time results_ecc = to_compute.compute()

In [ ]:
# we can save the results in a numpy array
np.save("results_ecc.npy", np.array([result.params for result in results_ecc]))

In [ ]:
# Now we can propagate the transformations throughout the whole stack.
results_ecc2 = list(accumulate(results_ecc, matmul))

In [ ]:
# and plot the results
fig, axs = plt.subplots(3,1, figsize=(3,9))
axs[0].plot(np.array([result.translation for result in results_ecc2]))
axs[1].plot(np.rad2deg(np.array([result.rotation for result in results_ecc2])))
axs[2].plot((np.array([result.scale for result in results_ecc2])))

- apply transforms
- materialization
- apply and bin and materialize